In [1]:
import pymysql
import math

import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

import os
import matplotlib.pyplot as plt

In [2]:
def connectdb():
    db = pymysql.connect(
        user = 'root',
        passwd = '110419',
        host = '127.0.0.1',
        db = 'knowledgebase',
        charset = 'utf8'
    )
    
    cursor = db.cursor(pymysql.cursors.DictCursor)
    
    return db, cursor

In [3]:
def separate_dict(data):
    names = []
    pathes = []
    
    for line in data:
        names.append(line['name'])
        
        path = line['path']
        path_split = path.split('\\')[:-1]
        pathes.append(path_split)
        
    return names, pathes

In [4]:
def E(p):
    return - math.log2(1 + p)
    #return math.log2(p) * p

In [5]:
def E2(p):
    #return - math.log2(1 + p)
    return math.log2(p) * p

In [6]:
def cal_entropy(name, path):
    db, cursor = connectdb()
    
    sql = "select distinct name, path from `exelist`;"
    cursor.execute(sql)
    result = cursor.fetchall()
    
    names, pathes = separate_dict(result)
    path = path.split('\\')[:-1]
    
    #name의 엔트로피
#    e = 0
#    for i in names:
#        if i == name:
#            p = 1 + 1/(names.count(name) + 1)
#        else:
#            p = 1
#        e += E(p)
#    name_e = 1 + e/math.log2(len(names))
    p = names.count(name)/len(names)
    name_e = E(p)

    #path의 각 단어를 반복문으로 돌면서 path의 등장 비율 계산하고 엔트로피계산해서 모두 더함    
    e = 0
    for i in path:
        tmp = 0
        total_i = 0
        
        pathes_tmp = sum(pathes, [])
        total_i = pathes_tmp.count(i)
        #for k in pathes:
        #    total_i += k.count(i)
            
        for j in pathes:
            p = 1 + j.count(i)/(total_i + 1)
            tmp += E2(p)
        e += tmp
    path_e = 1 + e/math.log2(len(pathes))
    
    return name_e+path_e
    #return name_e

In [7]:
def scoring(path):
    e_list = []
    infected_list = []
        
    df = pd.read_csv(path, index_col = 0)
    #df = df.drop_duplicates(["Name"])
    #df = df.reset_index(drop=True)
    #df = csv.drop_duplicates()
    #df = df.reset_index(drop = True)
    for idx in range(0, len(df)):
        name = df.loc[idx]["Name"]
        Path = df.loc[idx]["Path"]
            
        e = cal_entropy(name, Path)
        
        if df.loc[idx]["Infected"] == 1:
            infected_list.append(e)
        e_list.append(e)
            
    #e_list.sort(reverse = True)
    #for obj in infected_list:
    #    print(e_list.index(obj), " / ", len(csv))
        
    return e_list

In [8]:
normal = "normal_for_e.csv"
e_list_nm = scoring(normal)
normal_df = pd.DataFrame({'Entropy': e_list_nm})

min_entropy_nm = normal_df['Entropy'].min()

for i in range(0, len(normal_df)):
    if normal_df.loc[i]["Entropy"] == 0:
        normal_df.loc[i]["Entropy"] = min_entropy_nm
    
sorted_df_nm = normal_df.sort_values(by = 'Entropy', ascending = False)
sorted_df_nm = sorted_df_nm.reset_index(drop = True)

In [ ]:
path = "infected system/"
listdir = os.listdir(path)

for system in listdir:
    print("========================================================")
    print(system)
    e_list = scoring("is_after/" + system + "_wmi.csv")
    
    error_df = pd.DataFrame({'Entropy': e_list})
    
    
    min_entropy = error_df['Entropy'].min()

    for i in range(0, len(error_df)):
        if error_df.loc[i]["Entropy"] == 0:
            error_df.loc[i]["Entropy"] = min_entropy
    
    sorted_df = error_df.sort_values(by = 'Entropy', ascending = False)
    sorted_df = sorted_df.reset_index(drop = True)
    
    plt.clf()
    plt.plot(sorted_df[0:25], linestyle='--', marker = 'o', label=system, color = "red")
    plt.plot(sorted_df_nm[0:25], linestyle='--', marker = 'o', label="normal", color = "blue")
    plt.xlabel('Artifacts')
    plt.ylabel('Entropy')

    plt.legend()
    #plt.show()
    
    plt.savefig("result12/" + system + ".png")